## Plan Jan 07
### Look at evolution of topics over time (for leavers vs. remainers)
1. Identify topics (open-coding): LDA, PCA, network of terms. Also read comments / threads and apply my own labels.
2. Create "dictionaries" to map sets of terms to (potentially) theoretically interesting topics / codes.
3. Do this once overall, and next for over-lapping time-windows to see evolution of topics.

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk.sentiment.vader
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

//anaconda/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
dataset = pd.read_csv("csm subforums all posts cleaned.csv")

In [83]:
dataset.head(30)

,Unnamed: 0,ID,Subforum,subsectionpage,all_thread_ids,Thread_Title,Thread_Page,Date_of_post,Username,Post,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,1,1,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-04-19,Killer2,so we don't lose sight of all the summit topic...,...,"3,61","1,81","0,00","0,60","0,00","0,30","1,20","3,61","0,90","0,30"
1,2,2,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-04-21,Shepard Book,what happened to the post of the long list of ...,...,"0,00","0,00","0,00","4,17","0,00","0,00","0,00","0,00","0,00","0,00"
2,3,3,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-04-22,Olivia Ironsides,"csm5 was bad, fixing minor things/voting in th...",...,"4,55","0,00","0,00","0,00","0,00","0,00","0,00","0,00","9,09","4,55"
3,4,4,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-04-25,Consortium Agent,csm6 opted to forego player base consideration...,...,"0,00","1,27","0,00","0,00","0,00","0,00","0,00","2,53","2,53","0,00"
4,5,5,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-04-25,Killer2,"actually, you'll find that the topics we plan ...",...,"2,33","0,00","0,00","0,00","0,00","2,33","0,00","2,33","0,00","0,00"
5,6,6,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-04-26,Falin Whalen,"i sence much... hostility, in this one.prithee...",...,"5,71","0,00","0,00","2,86","0,00","0,00","0,00","0,00","0,00","0,00"
6,7,7,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-04-29,Consortium Agent,"odd, because i just listened to the fireside c...",...,"2,50","0,00","0,00","0,00","0,00","1,25","0,00","8,75","0,00","0,00"
7,8,8,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-04-29,Consortium Agent,i'm only hostile when the csm says one thing b...,...,"1,30","1,30","0,00","0,00","0,00","0,00","0,00","6,49","2,60","12,99"
8,9,9,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-05-01,Mr DurkaDur,"your totally right, 6 of the 7 summit topics ""...",...,"18,75","0,00","0,00","0,00","0,00","0,00","37,50","0,00","0,00","0,00"
9,10,10,Assembly Hall,1,1498941,[csm6] may summit topics thread index,1,2011-05-21,Hirana Yoshida,oy! will we get a preliminary report before re...,...,"6,03","0,00","0,00","0,86","4,31","6,03","0,00","0,00","1,72","4,31"


In [4]:
# posts = dataset['Post']

#### Select posts during the incarna crisis

In [5]:
# subset posts during the incarna crisis
# dataset['Date_of_post'] =  pd.to_datetime(dataset['Date_of_post'],infer_datetime_format=True)


In [6]:
# crisis_start = datetime.date(2011,3,1)
# crisis_end = datetime.date(2011,9,1)
# # (dataset['Date_of_post'].head() > crisis_start) & (dataset['Date_of_post'].head() < crisis_end)
# mask = (crisis_start <= dataset['Date_of_post']) & (dataset['Date_of_post'] <= crisis_end)
# posts_crisis = posts.loc[mask]
# print('posts during crisis = ' + str( len(posts_crisis)))
# posts_crisis.head()

Do topic analysis for threads, not posts.

In [85]:
dataset_sorted = dataset.sort_values(by=['all_thread_ids','Date_of_post']).reset_index(drop=True)
# dataset_sorted.head()

,Unnamed: 0,ID,Subforum,subsectionpage,all_thread_ids,Thread_Title,Thread_Page,Date_of_post,Username,Post,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,250172,262350,Assembly Hall,73,6339,hot topics of the assembly hall - aug 23 - 0.0...,1,2011-09-06,ccp spitfire,originally authored by obsidian hawk.ok so we ...,...,"4,14","5,80","0,00","1,10","0,83","2,21","0,00","0,55","0,00","2,21"
1,250173,262351,Assembly Hall,73,6339,hot topics of the assembly hall - aug 23 - 0.0...,1,2011-09-06,ccp spitfire,blasters 2.0blasters need a buffrailguns balan...,...,"1,00","0,50","0,00","0,00","1,99","3,98","0,00","0,50","0,00","5,97"
2,250174,262352,Assembly Hall,73,6339,hot topics of the assembly hall - aug 23 - 0.0...,1,2011-09-14,cyllan anassan,any chance of making cosmos missions expired t...,...,"5,41","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,250175,262353,Assembly Hall,73,6339,hot topics of the assembly hall - aug 23 - 0.0...,1,2011-09-16,samillian,bumpthis should be a sticky.,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,250176,262354,Assembly Hall,73,6339,hot topics of the assembly hall - aug 23 - 0.0...,1,2011-09-16,hirana yoshida,it was a sticky until someone pressed the wron...,...,"0,00","0,00","0,00","5,88","0,00","0,00","0,00","0,00","0,00","0,00"


In [110]:
# get all posts from each thread and concatenate them (Sorting by time)
# include columns for first post date, last post date, median post date
all_thread_ids = dataset_sorted['all_thread_ids'].unique()

thread_dict = {}
for thread_id in all_thread_ids:
    thread_items = dataset_sorted.loc[dataset_sorted["all_thread_ids"]==6339]
    first_date = min(thread_items["Date_of_post"])
    last_date =  max(thread_items["Date_of_post"])
    median_date = thread_items["Date_of_post"][round(len(thread_items)/2)]
    thread_posts = thread_items['Post'].str.cat(sep =" ---------------- ")
    thread_title = thread_items['Thread_Title'][0]
    first_poster = thread_items['Username'][0]
    most_freq_poster = thread_items['Username'].value_counts().idxmax()
    thread_dict[thread_id] = {'first_date':first_date,'last_date':last_date,'median_date':median_date,'thread_posts':thread_posts, 'thread_title':thread_title, 'first_poster':first_poster, 'most_freq_poster':most_freq_poster} 

In [111]:
threads = pd.DataFrame.from_dict(thread_dict)

### Remember: Change Tokenization to Phrases after proof-of-concept

In [7]:
# # Just checking what the tokenization looks like

# #tokens = nltk.word_tokenize(posts[0])
# post0 = posts_crisis[0]
# post0_sent = nltk.sentiment.vader.SentiText(posts_crisis[0])

In [8]:
# post0_sent.words_and_emoticons

In [9]:
# # writing the posts to an external file to be called by the VADER sentiment analyzer?

# file = open("posts_crisis.txt","w")
# file.write(posts[0])
# file.close()


In [10]:
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# analyser = SentimentIntensityAnalyzer()

In [11]:
# def print_sentiment_scores(sentence):
#     snt = analyser.polarity_scores(sentence)
    

In [12]:
# # test out the sentiment analyzer on the first 100 posts
# y = posts_crisis[0:100].apply(analyser.polarity_scores)
# y[0]

In [13]:
# N = len(posts_crisis)
# compound = [None for i in range(N)]
# negative = [None for i in range(N)]
# neutral = [None for i in range(N)]
# positive = [None for i in range(N)]

In [14]:
# # Very slow. Roughly 20 mins to process 318k posts @ 16k posts/min
# start = datetime.datetime.now()
# i = 0
# for post in posts_crisis: # note, can't index by i
    
#     try:
#         scores = analyser.polarity_scores(post)
#         compound[i] = scores["compound"]
#         negative[i] = scores["neg"]
#         neutral[i] = scores["neu"]
#         positive[i] = scores["pos"]
#     except:
#         print("exception at: "+str(i))
#     if i%1000 == 0:
#         print(str(i)+" posts   ||   time = "+str(datetime.datetime.now()-start))
#     i = i + 1
# print("Sentiment Analysis - Done.")

In [15]:
# sentiment_scores = pd.DataFrame({'compound':compound,'negative':negative,'neutral':neutral,'positive':positive})
# sentiment_scores.head()

In [16]:
# len(sentiment_scores) == len(posts_crisis)

In [17]:
# dataset_sent = dataset.loc[mask].join(sentiment_scores)
# dataset_sent.head()

In [18]:
# dataset_sent = dataset_sent.dropna()

In [19]:
# # dataset_sent.to_csv('posts_during_incarna_crisis_with_sentiment_scores.csv')
# dataset_sent = pd.read_csv('posts_during_incarna_crisis_with_sentiment_scores.csv')

In [20]:
# len(dataset_sent)
# len(dataset)

#### Aside: Visualization

In [21]:
# dataset2 = dataset_sent.groupby([pd.Grouper(key='Date_of_post', freq='W-MON')])['negative','positive'].mean().reset_index().sort_values('Date_of_post')


In [22]:
# window = 2

# ds2 = dataset2.dropna()
# ds3 = ds2.loc[:,ds2.columns!="Date_of_post"].rolling(window).mean().dropna()
# ds3 = ds3.join(ds2["Date_of_post"])

In [23]:
# plt.rcParams['figure.figsize'] = [10, 5]
# # plt.subplots(2, 2, sharex='col')

# title_txt = 'Sentiment Analysis of EVE Online Forumns'

# ds3.plot.line(x='Date_of_post',y=['positive','negative'], style=['g-','r-'])
# plt.title(title_txt)
# plt.ylabel('Sentiment Score \n('+str(window)+'-week averages)')
# plt.legend(loc=3)
# plt.grid()


# cname = "net_positive"
# ds4 = ds3
# ds4[cname] = ds3['positive']-ds3['negative']
# ds4.plot.line(x='Date_of_post',y=cname)
# plt.grid()
# plt.title(title_txt)
# plt.ylabel('Positive minus Negative sentiment scores \n('+ str(window) + '-week averages)')
# plt.legend(loc=3)

# # plt.show()

### Select comments that match the date and rate above some threshold for negativity on sentiment


In [24]:
# dsneg = dataset_sent.sort_values("negative",ascending=False)
# for post in dsneg["Post"].head():
#     print(post)
#     print()

#### Commentary:
Looking at this post at least, there does seem to be some negative sentiment or at least somewhat raised emotion. Let's move forward with the topic analysis.

Why not just start with topic analysis (or Nomological Network Analysis) and see which topics are associated with negativity? Maybe we will re-discover the mechanics behind the Sentiment analyzer, but maybe we will see some interesting stuff too.

### Topic Modeling

In [25]:
# From https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
# didn't have gensim pre-loaded so went to terminal and installed it with $pip install gensim

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nicholas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Write a function to perform lemmatize and stem preprocessing steps on the data set.



In [26]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    # using a try block here because there are some cases where this fails (I guess for empty posts)
    try: 
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
                result.append(lemmatize_stemming(token))
    except:
        pass
    return result

Select a document to preview after preprocessing.



In [27]:
documents = dataset['Post']
documents.head()

0    so we don't lose sight of all the summit topic...
1    what happened to the post of the long list of ...
2    csm5 was bad, fixing minor things/voting in th...
3    csm6 opted to forego player base consideration...
4    actually, you'll find that the topics we plan ...
Name: Post, dtype: object

In [28]:
doc_sample = documents[0]
print('original document: ')
print(doc_sample)
words = []
for word in doc_sample.split(' '):
    words.append(word)
# print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
so we don't lose sight of all the summit topic threads, here's an index with a short summary about each one.team bff & the thousand little things.:in this thread, we welcome the community to offer suggestions about how csm can help team bff focus their efforts to get even more "bang for buck". we are not just looking for specific little things that annoy you and that you'd like fixed, but also suggestions about improving the process of identifying and prioritizing them.ship balance:in this thread, we're looking for everything relating to ship balancing, encompassing everything from individual ships to how they interact in fleets. the sky is the limit.nullsec industrialization ("farms and fields"):in this thread, we're looking for advice and feedback on how to improve nullsec industrialization. making nullsec less dependent on jita for everything, providing farms and fields for small-gang targets beyond sov structures. this may degenerate into a 'for gods sakes imple

#### commentary: 
We might not what to use stemming since it obscures what is being talked about and leaves out local-specific language (like 'afk' and 'ccp' and 'csm'). But for now we can live with it.

Preprocess the posts, saving the results as ‘processed_docs’   

In [29]:
processed_docs = documents.map(preprocess)
processed_docs[0]

['lose',
 'sight',
 'summit',
 'topic',
 'thread',
 'index',
 'short',
 'summari',
 'team',
 'thousand',
 'littl',
 'thing',
 'thread',
 'welcom',
 'communiti',
 'offer',
 'suggest',
 'help',
 'team',
 'focus',
 'effort',
 'bang',
 'buck',
 'look',
 'specif',
 'littl',
 'thing',
 'annoy',
 'like',
 'fix',
 'suggest',
 'improv',
 'process',
 'identifi',
 'priorit',
 'ship',
 'balanc',
 'thread',
 'look',
 'relat',
 'ship',
 'balanc',
 'encompass',
 'individu',
 'ship',
 'interact',
 'fleet',
 'limit',
 'nullsec',
 'farm',
 'field',
 'thread',
 'look',
 'advic',
 'feedback',
 'improv',
 'nullsec',
 'make',
 'nullsec',
 'depend',
 'jita',
 'provid',
 'farm',
 'field',
 'small',
 'gang',
 'target',
 'structur',
 'degener',
 'god',
 'sake',
 'implement',
 'miss',
 'dominion',
 'featur',
 'discuss',
 'custom',
 'support',
 'thread',
 'look',
 'feedback',
 'way',
 'improv',
 'custom',
 'support',
 'focus',
 'polici',
 'procedur',
 'exist',
 'individu',
 'petit',
 'detail',
 'player',
 'experi

In [30]:
# may try to add bigrams at a later point. too slow now.
# import re
# l=0
# # add bigrams and trigrams to the dictionary.
# for i in range(len(processed_docs)):
#     unigrams = processed_docs[i]
#     bigrams = nltk.bigrams(unigrams)
#     for bigram in bigrams:
#         processed_docs[i].append(" ".join(bigram))


##### Bag of Words on the Data set
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [31]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

72505 battlecrab
61647 minz
31397 vaction
33317 vekarra
67032 wookie
9617 degrad
55021 unbl
16120 circul
72220 ienergi
90940 deil
94324 tigerit


##### Filter out tokens that appear in
- less than 50 posts (absolute number) or
- more than 50% of documents (fraction of total corpus size, not absolute number).
- after the above two steps, keep only the first 100,000 most frequent tokens.

In [32]:
old_dictionary = dictionary

In [33]:

print("old len: "+ str(len(dictionary)))
dictionary.filter_extremes(no_below=50, no_above=0.50, keep_n=100000)
print("new len: "+ str(len(dictionary)))


old len: 116323
new len: 6413


For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’ (bow = bag of words), then check our selected document earlier.

In [34]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# bow_corpus[0]

look at the bag of words representation of the first document

In [35]:
# bow_doc_0 = bow_corpus[0]
# for i in range(len(bow_doc_0)):
#     print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
#                                                dictionary[bow_doc_0[i][0]], 
# bow_doc_0[i][1]))

Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

TFIDF normalizes the counts of a word in each document by the word's relative rarity in the corpus. Therefore, common words are given lower wieght, and rare words are given more weight in each document.

In [36]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break


[(0, 0.17404072393243772),
 (1, 0.065969816087878136),
 (2, 0.098403959147352865),
 (3, 0.10026394901522014),
 (4, 0.11005797551042822),
 (5, 0.099034496230123989),
 (6, 0.056698060606799223),
 (7, 0.0418135070909888),
 (8, 0.13400773195241139),
 (9, 0.118235407374384),
 (10, 0.065215311947906945),
 (11, 0.06919979016853374),
 (12, 0.09876103711097324),
 (13, 0.081207615246209286),
 (14, 0.11811343459057873),
 (15, 0.11500626350574748),
 (16, 0.052924863402011864),
 (17, 0.053952513026636582),
 (18, 0.15079993026441824),
 (19, 0.056414071689049584),
 (20, 0.21181091027573931),
 (21, 0.048199841383928958),
 (22, 0.1276347516947508),
 (23, 0.053425457384577095),
 (24, 0.047857364617419594),
 (25, 0.12198233200071013),
 (26, 0.053220786122736716),
 (27, 0.090794759664156371),
 (28, 0.056651552799433326),
 (29, 0.07688459423730995),
 (30, 0.043313159352001043),
 (31, 0.1011473487460975),
 (32, 0.033108702838659805),
 (33, 0.047962673794692393),
 (34, 0.046529024002448106),
 (35, 0.12217357

##### Running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [37]:
n_topics = 100
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=n_topics, id2word=dictionary, passes=2, workers=2)


//anaconda/lib/python3.5/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Note the number of topics that were chosen above. Using more topics can discover more fine-grained and interesting themes, but also has a chance at discovering words that seemed to clump together just due to noise.

For each topic, we will explore the words occuring in that topic and its relative weight.

In [38]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print()

Topic: 0 
Words: 0.188*"thank" + 0.123*"hope" + 0.049*"chairman" + 0.046*"comment" + 0.041*"address" + 0.039*"that" + 0.019*"fault" + 0.018*"recommend" + 0.014*"fund" + 0.012*"virtual"

Topic: 1 
Words: 0.151*"goon" + 0.051*"mitten" + 0.025*"social" + 0.023*"trust" + 0.019*"announc" + 0.019*"individu" + 0.016*"think" + 0.016*"conflict" + 0.016*"feedback" + 0.015*"resourc"

Topic: 2 
Words: 0.125*"item" + 0.055*"folk" + 0.055*"store" + 0.047*"pain" + 0.046*"dead" + 0.037*"model" + 0.029*"document" + 0.023*"opportun" + 0.022*"flip" + 0.022*"wear"

Topic: 3 
Words: 0.107*"sorri" + 0.057*"abus" + 0.046*"unit" + 0.036*"laugh" + 0.031*"somebodi" + 0.028*"realiti" + 0.028*"straight" + 0.025*"fals" + 0.022*"harass" + 0.021*"door"

Topic: 4 
Words: 0.105*"develop" + 0.083*"voic" + 0.065*"process" + 0.057*"wasn" + 0.051*"cool" + 0.036*"code" + 0.028*"influenc" + 0.020*"zero" + 0.019*"approach" + 0.018*"work"

Topic: 5 
Words: 0.098*"warp" + 0.062*"rare" + 0.056*"silent" + 0.046*"easi" + 0.028*"r

Running LDA using TF-IDF



In [39]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=100, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.023*"steal" + 0.021*"transpar" + 0.018*"land" + 0.017*"suspect" + 0.017*"amber" + 0.015*"dismiss" + 0.014*"loot" + 0.014*"tell" + 0.013*"mutual" + 0.013*"playerbas"
Topic: 1 Word: 0.012*"surpris" + 0.009*"aggro" + 0.009*"game" + 0.008*"player" + 0.008*"protest" + 0.008*"guid" + 0.008*"chicken" + 0.007*"adapt" + 0.006*"peopl" + 0.006*"morn"
Topic: 2 Word: 0.028*"request" + 0.025*"congratul" + 0.022*"amaz" + 0.016*"moron" + 0.016*"squad" + 0.014*"monocl" + 0.012*"integr" + 0.011*"bold" + 0.010*"connect" + 0.010*"colour"
Topic: 3 Word: 0.034*"want" + 0.034*"killmail" + 0.022*"skin" + 0.019*"iron" + 0.018*"tutori" + 0.016*"weird" + 0.015*"sub" + 0.015*"finger" + 0.015*"middl" + 0.014*"kitti"
Topic: 4 Word: 0.020*"ship" + 0.020*"cyno" + 0.017*"frigat" + 0.016*"cruiser" + 0.014*"drink" + 0.011*"battleship" + 0.011*"covert" + 0.010*"disappoint" + 0.010*"revamp" + 0.010*"round"
Topic: 5 Word: 0.063*"skill" + 0.052*"train" + 0.033*"learn" + 0.017*"charact" + 0.017*"time" + 0.01

//anaconda/lib/python3.5/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


This doesn't seem to be super useful in itself. It might be more useful to map it back to the comments and look for the most "representative" comments.

In [40]:
print(documents[0])
print(processed_docs[0])
print()
print(documents.loc[1])
print(processed_docs[1])
print()


so we don't lose sight of all the summit topic threads, here's an index with a short summary about each one.team bff & the thousand little things.:in this thread, we welcome the community to offer suggestions about how csm can help team bff focus their efforts to get even more "bang for buck". we are not just looking for specific little things that annoy you and that you'd like fixed, but also suggestions about improving the process of identifying and prioritizing them.ship balance:in this thread, we're looking for everything relating to ship balancing, encompassing everything from individual ships to how they interact in fleets. the sky is the limit.nullsec industrialization ("farms and fields"):in this thread, we're looking for advice and feedback on how to improve nullsec industrialization. making nullsec less dependent on jita for everything, providing farms and fields for small-gang targets beyond sov structures. this may degenerate into a 'for gods sakes implement the missing dom

In [41]:
print(documents.loc[0][0])
print(processed_docs[0])
for index, score in sorted(lda_model_tfidf[bow_corpus[0]], key=lambda tup: -1*tup[1])[:3]:
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

s
['lose', 'sight', 'summit', 'topic', 'thread', 'index', 'short', 'summari', 'team', 'thousand', 'littl', 'thing', 'thread', 'welcom', 'communiti', 'offer', 'suggest', 'help', 'team', 'focus', 'effort', 'bang', 'buck', 'look', 'specif', 'littl', 'thing', 'annoy', 'like', 'fix', 'suggest', 'improv', 'process', 'identifi', 'priorit', 'ship', 'balanc', 'thread', 'look', 'relat', 'ship', 'balanc', 'encompass', 'individu', 'ship', 'interact', 'fleet', 'limit', 'nullsec', 'farm', 'field', 'thread', 'look', 'advic', 'feedback', 'improv', 'nullsec', 'make', 'nullsec', 'depend', 'jita', 'provid', 'farm', 'field', 'small', 'gang', 'target', 'structur', 'degener', 'god', 'sake', 'implement', 'miss', 'dominion', 'featur', 'discuss', 'custom', 'support', 'thread', 'look', 'feedback', 'way', 'improv', 'custom', 'support', 'focus', 'polici', 'procedur', 'exist', 'individu', 'petit', 'detail', 'player', 'experi', 'thread', 'discuss', 'great', 'stride', 'improv', 'overal', 'like', 'gather', 'feedback'

In [52]:
import random
round(len(bow_corpus)*.10)

31866

In [81]:
# # very slow. Randomly select documents to label.
# sample_fraction = 0.001 # sample as a percent of all posts
# sample_size = round(len(bow_corpus)*sample_fraction)


# # n_topics = 100


# # map the model to posts
# a = lda_model_tfidf.get_document_topics(bow_corpus[0],minimum_probability=0)

# docs_LDA_scores = pd.DataFrame(columns = range(n_topics))

# d = dict(a)
# i=0
# start = datetime.datetime.now()
# for i in random.sample(range(len(bow_corpus)),sample_size):
#     doc = bow_corpus[i]
#     topics_of_doc = lda_model_tfidf.get_document_topics(doc,minimum_probability=0.00)
#     v = np.array(topics_of_doc)[:,1]
#     row = pd.DataFrame(v).transpose()
#     docs_LDA_scores = docs_LDA_scores.append(row)
    
#     if i%1000 == 0:
#         print(str(i)+" posts   ||   time = "+str(datetime.datetime.now()-start))
#         docs_LDA_scores.to_csv('docs_LDA_scores.csv')
#     i = i + 1
# print("LDA Mapping - Done.")

In [82]:
# i = random.randint(0,31000)
# print( bow_corpus[i])
# print(documents[i])
# print(dataset.loc[i])

In [ ]:
# docs_LDA_scores.index = range(len(docs_LDA_scores))

# show_n_comments = 20


# for topic_num in range(n_topics):
# # for topic_num in range(3,12):
#     top_docs_idx = docs_LDA_scores.sort_values(topic_num,ascending=False)[topic_num][:show_n_comments].index
#     print("Topic: "+str(topic_num))
#     print(lda_model_tfidf.print_topic(topic_num, 15))
#     print()
#     print('Comments:')
#     for idx in top_docs_idx:
#         print("[] " + documents.loc[idx][0])
#     print()

As expected, LDA doesn't seem to be super useful. It might be more useful with entire threads, not just individual comments. Let's look at PCA / lexical networks next.

## Looking at PCA 
Just cluster / PCA by TFIDF

In [ ]:
from sklearn.decomposition import PCA #, FastICA

# corpus_tfidf_df = pd.DataFrame(corpus_tfidf)
corpus_tfidf_csr = gensim.matutils.corpus2csc(corpus_tfidf)
corpus_tfidf_numpy = corpus_tfidf_csr.T.toarray()

# corpus_tfidf_array = corpus_tfidf.T.toarray()

In [ ]:
pca_tool = PCA(n_components=200)
pca_tool.fit(corpus_tfidf_numpy)

# ICA was too slow.
# ica = FastICA(n_components=200)
# ica.fit(corpus_tfidf_numpy)

In [ ]:
transformed_corpus = pca_tool.transform(corpus_tfidf_numpy)


In [ ]:
transformed_corpus.shape

In [ ]:
# rng = np.random.RandomState(1)
X = transformed_corpus[:,[0,1]]
plt.scatter(X[:, 0], X[:, 1])
plt.title('Comments projected into top 2 dimensions');

In [ ]:
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal');
plt.title('Comments projected into top 2 dimensions');

In [ ]:
pca_tool.components_.shape

In [ ]:
prin_comps = pd.DataFrame(pca_tool.components_)
prin_comps

In [ ]:
for j in range(5):
    
    print("Component numer: " + str(j))

    component_j = prin_comps[j]

    order = component_j.map(lambda x : x).abs().sort_values(ascending = False)
    component_j[order.index]


    for i in range(10):
        idx = order.index[i]
        word = dictionary[idx]
        weight = str(component_j[idx])
        print(weight + " : " + word)
        
    print()

# sns.distplot(S);

These don't seem to be super interpretable. Try selecting representative comments.

In [ ]:
transformed_corpus_df = pd.DataFrame(transformed_corpus)


# for the top component, select the indexes that are most associated with this comp and read them
for comp_num in range(100):
# comp_num = 0
    print("Component numer: " + str(comp_num))
    sorted_df = transformed_corpus_df.sort_values(by=comp_num,ascending=False)
    for top_comment_num in range(10):
    #     top_comment_num = 0
        top_comment_idx = sorted_df.index[top_comment_num]
        print(documents.loc[top_comment_idx][0])
        print()
# order = component_j.map(lambda x : x).abs().sort_values(ascending = False)

# component_j[order.index]

These seem to be more interpretable. The components seem to make more sense, although in some cases it is clear they are leaning heavily on certain words. 

Now let's go back to the network to see what emerges.


Let's do a quick regression to see which components align with negativity

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
X = transformed_corpus
y = list(documents['negative'])

results = sm.OLS(y, X).fit()
print(results.summary())

Now let's go back to the network to see what emerges.

### Clustering
#### Hierarchical Clustering
PCA already reduced the dimensionality, so Hierarchical Clustering should not be too too slow.


### Lexical Networks
Plan:
1. Add bigrams and trigrams to the dictionary (can do later)
    2. Then filter it again for low-frequency terms or super high-freq terms (>50%)
3. Create an M x M matrix, where M is the number of terms in the dictionary
4. O(N) to fill out the matrix. Iterate through the list of comments (That have been reduced to bag of words/terms). In each post, for each pair of words, increment the value in the matrix, weighted by TFIDF (the product of the two terms' value).
5. Once done, make the matrix sparser by thresholding values below the 50% (median) value.
6. Visualize the matrix (probably get a hairball). Can tune the thresholding until getting cleaner separation.
7. Run community detection algo to get 'themes'

In [ ]:
# don't even need to create a matrix (too much space). Just create a dictionary (link_list)
idx = 0
link_weights = {}
link_text = {}
for post in corpus_tfidf:
    L = len(post)
    # cycle thru each word (i) in the post and the subsequent words (j).
    # if this pair of words is unseen, add it to the dict of weights. Otherwise, add it to the existing entry
    for i in range(L):
        for j in range(i,L):
            if (i<j):
                weight = post[i][1]*post[j][1]
#                 print(str(i) + " , " + str(j) + ": " + str(weight))
                
                the_key = (dictionary[post[i][0]],dictionary[post[j][0]])
                if the_key in link_weights:
                    link_weights[the_key] = link_weights[the_key]+ weight
#                     link_text[the_key].append(documents.loc[idx][0])
                else:
                    link_weights[the_key] = weight
                    link_text[the_key] = [documents.loc[idx][0]]
    if idx % 500 == 0:
        print(str(idx)+" posts   ||   date-time = "+str(datetime.datetime.now()))
    idx = idx + 1
print('Done.')
                    

In [ ]:

# link_weights.keys()
# link_weights.values()
k = list(link_weights.keys())
v = list(link_weights.values())
d = {'links': k, 'weights': v}

link_weights_df = pd.DataFrame(d)
link_weights_df.sort_values(by='weights',ascending=False)


In [ ]:
len(link_weights_df)

The "strongest" link is (item, vaniti)	

Visualize the distribution:

In [ ]:
import seaborn as sns
#Log Scale
sns.distplot(np.log(v));


Select only the links with strong weights

In [ ]:
order = link_weights_df.sort_values(by='weights',ascending=False).index

#### How many links? top 100

In [ ]:
# top_pct = 0.10
# top_N = top_pct*len(order)
top_N = 100# just pick 100 arbitrarily
top_links = order[:top_N]
top_link_weights_df = link_weights_df.loc[top_links]
# top_link_weights_df

In [ ]:
# need to split the links into separate columns after all

top_link_weights_df[['node_i','node_j']] = top_link_weights_df['links'].apply(pd.Series)
top_link_weights_df

#### Creating a Graph

In [ ]:
import networkx as nx

In [ ]:
# Creating a Graph 
G=nx.from_pandas_dataframe(top_link_weights_df, 'node_i', 'node_j', ['weights'])

In [ ]:
nx.draw_spectral(G)

Spectral representation looks like it structures the nodes well. Maybe try spectral clustering on the network to find the key themes.

In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn import metrics

n_clusters = 8

# Get adjacency-matrix as numpy-array
adj_mat = nx.to_numpy_matrix(G,weight='weights')

# Cluster
sc = SpectralClustering(n_clusters, affinity='precomputed', n_init=100)
sc.fit(adj_mat)


In [ ]:
adj_mat.max()

In [ ]:
plt.hist(sc.labels_,normed=True,bins=n_clusters); plt.title('spectral clustering')

In [ ]:
# Try another clustering algo
# from networkx.algorithms.community import greedy_modularity_communities
# import community

In [ ]:
#first compute the best partition
# partition = community.best_partition(G)

In [ ]:
pagerank = nx.eigenvector_centrality_numpy(G)
# k_pr = list(pagerank.keys())
# v_pr = list(pagerank.values())
# df_pr = pd.DataFrame({'term' : k_pr, 'eigen_centrality':v_pr})
# pagerank

In [ ]:
df = pd.DataFrame({'term' : G.nodes(), 'cluster' : sc.labels_})
df['eigen_centrality'] = 0
for idx in df.index:
    term = df.loc[idx,'term']
    df.loc[idx,'eigen_centrality'] = pagerank[term]
df.sort_values('eigen_centrality',ascending=False)

In [ ]:
# Create-subgraphs and visualize each one.
top_link_weights_df[['node_i','node_j']]

# df.sort_values(by='cluster')

sub_community = {}
for clust_id in range(n_clusters):
    community_nodes = list(df[df['cluster']==clust_id]['term'])
    sub_community[clust_id] = top_link_weights_df[top_link_weights_df['node_i'].isin(community_nodes)]
    print(df[df['cluster']==clust_id].sort_values('eigen_centrality',ascending=False))
    print()

# for each term, get the top weighted link
# for each link, get a comment with that link in it (this might take more time, should do ahead of time)

In [ ]:
plt.figure(3,figsize=(12,12)) 
nx.draw(G,with_labels=True,node_size=4000*df['eigen_centrality'],node_color=sc.labels_,font_size=15)

In [ ]:
clust_id = 1
sub_G=nx.from_pandas_dataframe(sub_community[clust_id], 'node_i', 'node_j', ['weights'])
sub_community[clust_id]
community_nodes = list(df[df['cluster']==clust_id]['term'])
in_commnity =  pd.Series(sub_G.nodes()).isin(community_nodes)

df2 = pd.DataFrame({'term' : sub_G.nodes()})
df2['eigen_centrality'] = 0
for idx in df2.index:
    term = df2.loc[idx,'term']
    df2.loc[idx,'eigen_centrality'] = pagerank[term]
# df2.sort_values('eigen_centrality',ascending=False)
# larger figure size
plt.figure(3,figsize=(12,12)) 
nx.draw(sub_G,with_labels=True,node_size=10*in_commnity,node_color=in_commnity*2,cmap=plt.get_cmap('rainbow'),font_size=10)
plt.show()

In [ ]:
# larger figure size
plt.figure(3,figsize=(12,12)) 
nx.draw(sub_G,with_labels=True,node_size=10*in_commnity,node_color=in_commnity*2,cmap=plt.get_cmap('rainbow'),font_size=10)
plt.show()


In [ ]:
clust_id

#### How many links? top 1000

In [ ]:
# top_pct = 0.10
# top_N = top_pct*len(order)
top_N = 1000# just pick 1000 arbitrarily
top_links = order[:top_N]
top_link_weights_df = link_weights_df.loc[top_links]
# top_link_weights_df

In [ ]:
# need to split the links into separate columns after all

top_link_weights_df[['node_i','node_j']] = top_link_weights_df['links'].apply(pd.Series)
top_link_weights_df

#### Creating a Graph

In [ ]:
import networkx as nx

In [ ]:
# Creating a Graph 
G=nx.from_pandas_dataframe(top_link_weights_df, 'node_i', 'node_j', ['weights'])

In [ ]:
# nx.draw_spectral(G)

Spectral representation looks like it structures the nodes well. Maybe try spectral clustering on the network to find the key themes.

In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn import metrics

n_clusters = 50

# Get adjacency-matrix as numpy-array
adj_mat = nx.to_numpy_matrix(G,weight='weights')

# Cluster
sc = SpectralClustering(n_clusters, affinity='precomputed', n_init=100)
sc.fit(adj_mat)


In [ ]:
adj_mat.max()

In [ ]:
# plt.hist(sc.labels_,normed=True,bins=n_clusters); plt.title('spectral clustering')

In [ ]:
# Try another clustering algo
# from networkx.algorithms.community import greedy_modularity_communities
# import community

In [ ]:
#first compute the best partition
# partition = community.best_partition(G)

In [ ]:
pagerank = nx.eigenvector_centrality_numpy(G)
# k_pr = list(pagerank.keys())
# v_pr = list(pagerank.values())
# df_pr = pd.DataFrame({'term' : k_pr, 'eigen_centrality':v_pr})
# pagerank

In [ ]:
df = pd.DataFrame({'term' : G.nodes(), 'cluster' : sc.labels_})
df['eigen_centrality'] = 0
for idx in df.index:
    term = df.loc[idx,'term']
    df.loc[idx,'eigen_centrality'] = pagerank[term]
df.sort_values('eigen_centrality',ascending=False)

In [ ]:
# Create-subgraphs and visualize each one.
top_link_weights_df[['node_i','node_j']]

# df.sort_values(by='cluster')

sub_community = {}
for clust_id in range(n_clusters):
    community_nodes = list(df[df['cluster']==clust_id]['term'])
    sub_community[clust_id] = top_link_weights_df[top_link_weights_df['node_i'].isin(community_nodes)]
    print(df[df['cluster']==clust_id].sort_values('eigen_centrality',ascending=False))
    print()

# for each term, get the top weighted link
# for each link, get a comment with that link in it (this might take more time, should do ahead of time)

In [ ]:
plt.figure(3,figsize=(12,12)) 
nx.draw(G,with_labels=True,node_size=4000*df['eigen_centrality'],node_color=sc.labels_,font_size=15)

In [ ]:
clust_id = 17
sub_G=nx.from_pandas_dataframe(sub_community[clust_id], 'node_i', 'node_j', ['weights'])
sub_community[clust_id]
community_nodes = list(df[df['cluster']==clust_id]['term'])
in_commnity =  pd.Series(sub_G.nodes()).isin(community_nodes)

df2 = pd.DataFrame({'term' : sub_G.nodes()})
df2['eigen_centrality'] = 0
for idx in df2.index:
    term = df2.loc[idx,'term']
    df2.loc[idx,'eigen_centrality'] = pagerank[term]
# df2.sort_values('eigen_centrality',ascending=False)
# larger figure size
plt.figure(3,figsize=(12,12)) 
nx.draw(sub_G,with_labels=True,node_size=1500*in_commnity,node_color=in_commnity*2,cmap=plt.get_cmap('rainbow'),font_size=12)
plt.show()

In [ ]:
# get sub-clusters from cluster 0
clust_id = 0
sub_G=nx.from_pandas_dataframe(sub_community[clust_id], 'node_i', 'node_j', ['weights'])

In [ ]:
# larger figure size
plt.figure(3,figsize=(12,12)) 
nx.draw(sub_G,with_labels=True,node_size=10*in_commnity,node_color=in_commnity*2,cmap=plt.get_cmap('rainbow'),font_size=10)
plt.show()


In [ ]:
clust_id

In [ ]:
plt.figure(3,figsize=(12,12)) 
nx.draw(G,with_labels=True,node_size=1000*df['eigen_centrality'],node_color=sc.labels_,font_size=10)

#### How many links? top 600

In [ ]:
# top_pct = 0.10
# top_N = top_pct*len(order)
top_N = 600# just pick 1000 arbitrarily
top_links = order[:top_N]
top_link_weights_df = link_weights_df.loc[top_links]
# top_link_weights_df

In [ ]:
# need to split the links into separate columns after all

top_link_weights_df[['node_i','node_j']] = top_link_weights_df['links'].apply(pd.Series)
top_link_weights_df

#### Creating a Graph

In [ ]:
import networkx as nx

In [ ]:
# Creating a Graph 
G=nx.from_pandas_dataframe(top_link_weights_df, 'node_i', 'node_j', ['weights'])

In [ ]:
nx.draw_spectral(G)

Spectral representation looks like it structures the nodes well. Maybe try spectral clustering on the network to find the key themes.

In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn import metrics

n_clusters = 8

# Get adjacency-matrix as numpy-array
adj_mat = nx.to_numpy_matrix(G,weight='weights')

# Cluster
sc = SpectralClustering(n_clusters, affinity='precomputed', n_init=100)
sc.fit(adj_mat)


In [ ]:
adj_mat.max()

In [ ]:
plt.hist(sc.labels_,normed=True,bins=n_clusters); plt.title('spectral clustering')

In [ ]:
# Try another clustering algo
# from networkx.algorithms.community import greedy_modularity_communities
# import community

In [ ]:
#first compute the best partition
# partition = community.best_partition(G)

In [ ]:
pagerank = nx.eigenvector_centrality_numpy(G)
# k_pr = list(pagerank.keys())
# v_pr = list(pagerank.values())
# df_pr = pd.DataFrame({'term' : k_pr, 'eigen_centrality':v_pr})
# pagerank

In [ ]:
df = pd.DataFrame({'term' : G.nodes(), 'cluster' : sc.labels_})
df['eigen_centrality'] = 0
for idx in df.index:
    term = df.loc[idx,'term']
    df.loc[idx,'eigen_centrality'] = pagerank[term]
df.sort_values('eigen_centrality',ascending=False)

In [ ]:
# Create-subgraphs and visualize each one.
top_link_weights_df[['node_i','node_j']]

# df.sort_values(by='cluster')

sub_community = {}
for clust_id in range(n_clusters):
    community_nodes = list(df[df['cluster']==clust_id]['term'])
    sub_community[clust_id] = top_link_weights_df[top_link_weights_df['node_i'].isin(community_nodes)]
    print(df[df['cluster']==clust_id].sort_values('eigen_centrality',ascending=False))
    print()

# for each term, get the top weighted link
# for each link, get a comment with that link in it (this might take more time, should do ahead of time)

In [ ]:
clust_id = 1
sub_G=nx.from_pandas_dataframe(sub_community[clust_id], 'node_i', 'node_j', ['weights'])
sub_community[clust_id]
community_nodes = list(df[df['cluster']==clust_id]['term'])
in_commnity =  pd.Series(sub_G.nodes()).isin(community_nodes)

df2 = pd.DataFrame({'term' : sub_G.nodes()})
df2['eigen_centrality'] = 0
for idx in df2.index:
    term = df2.loc[idx,'term']
    df2.loc[idx,'eigen_centrality'] = pagerank[term]
# df2.sort_values('eigen_centrality',ascending=False)
# larger figure size
plt.figure(3,figsize=(12,12)) 
nx.draw(sub_G,with_labels=True,node_size=10*in_commnity,node_color=in_commnity*2,cmap=plt.get_cmap('rainbow'),font_size=10)
plt.show()

In [ ]:
# larger figure size
plt.figure(3,figsize=(12,12)) 
nx.draw(sub_G,with_labels=True,node_size=10*in_commnity,node_color=in_commnity*2,cmap=plt.get_cmap('rainbow'),font_size=10)
plt.show()


In [ ]:
clust_id

In [ ]:
plt.figure(3,figsize=(12,12)) 
nx.draw(G,with_labels=True,node_size=1000*df['eigen_centrality'],node_color=sc.labels_,font_size=10)

In [ ]:
nx.draw_spring(G,with_labels=True,node_size=1000*df['eigen_centrality'],node_color=sc.labels_)

In [ ]:
nx.draw_spectral(G,node_size=100,node_color=sc.labels_)

In [ ]:
# given: list of nodes (N), and list of edges E=(u,v), get the list of edges that intersects with nodes in N
top_link_weights_df[top_link_weights_df['node_i'].isin(community_nodes)]

In [ ]:
which = lambda lst:list(np.where(lst)[0])


# community_nodes = ['transact', 'micro']
# ni = top_link_weights_df['node_i']
# nj = top_link_weights_df['node_j']
# matches = [True for x in ni if x in community_nodes]
# matches

In [ ]:
documents.loc[idx][0]

# Next Steps
1. Look more at the forumns / stuff qualitatively
2. Develop a qualitative nomological network
3. perhaps use some emergent constructs from the findings above (e.g. types of posts, themes, etc.) but be very open to seeing new constructs that cannot be seen directly from the vocab (e.g. power, old-guard, crowd, etc.)
4. Think about how to integrate emerging ideas & constructs with this text analysis. 
    1. Filtering (e.g. by key-word)
    2. By Negativity?
    3. Modeling? PCA - negativity?
    